In [15]:
from scipy.linalg import expm
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis,spinless_fermion_basis_1d,boson_basis_1d, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [16]:
L = 3
M = 0.1
l=1
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [17]:
basis_spin =spin_basis_1d(L=L,S = str(l))
basis_fermions = spinless_fermion_basis_1d(L=L+1,Nf = [0,1,2,3,4])
basis=tensor_basis(basis_spin,basis_fermions)
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
	  0.         |2 2 2>           26         |1 1 1 1>           15  
	  1.         |2 2 2>           26         |1 1 1 0>           14  
	  2.         |2 2 2>           26         |1 1 0 1>           13  
	  3.         |2 2 2>           26         |1 1 0 0>           12  
	  4.         |2 2 2>           26         |1 0 1 1>           11  
	  5.         |2 2 2>           26         |1 0 1 0>           10  
	  6.         |2 2 2>           26         |1 0 0 1>            9  
	  7.         |2 2 2>           26         |1 0 0 0>            8  
	  8.         |2 2 2>           26         |0 1 1 1>            7  
	  9.         |2 2 2>           26         |0 1 1 0>            6  
	 10.         |2 2 2>           26         |0 1 0 1>            5  
	 11.         |2 2 2>           26         |0 1 0 0>            4  
	 12.         |2 2 2>           26         |0 0 1 1>            3  
	 13.         |2 2 2>           26         |0 0

In [18]:
basis_vectors = []
for index in range(basis.Ns):
    state = np.zeros(basis.Ns)
    state[basis.Ns-index-1] = 1.
    state = state.tolist()
    basis_vectors.append(state)

In [19]:
#initialisation of the Gauss laws

liste = []
liste_m = []
gauss_law_map = []
Gauss_law = []

for i in range(L+1):
    liste.append([[1.,i]])
    liste_m.append([[-1.,i]])
    if i == 0:
        gauss_law_map.append([
            ["z|",liste[0]],
            ["|n",liste_m[0]],
        ])
    
    elif (i == L) & (i%2 == 0):
        gauss_law_map.append([
            ["z|",liste_m[i-1]],
            ["|n",liste_m[i]]
        ])
    
    elif (i == L) & (i%2 == 1):
        gauss_law_map.append([
            ["z|",liste_m[i-1]],
            ["I|",liste[i-1]],
            ["|n",liste_m[i]],
        ])        
        
    elif i%2 == 0:
        gauss_law_map.append([
            ["z|",liste[i]],
            ["z|",liste_m[i-1]],
            ["|n",liste_m[i]],
        ])
    elif i%2 == 1:
        gauss_law_map.append([
            ["z|",liste[i]],
            ["z|",liste_m[i-1]],  
            ["I|",liste[i]],
            ["|n",liste_m[i]],
        ])
    Gauss_law.append(hamiltonian(gauss_law_map[i],dynamic_list=[],basis=basis,**no_checks))
G = 0
for i in range(L+1):
    G += Gauss_law[i]**2

In [20]:
kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[M*(-1)**i,i] for i in range(L+1)]
interaction_term = [[-1j/(2*np.sqrt(l*(l+1))),i,i,i+1] for i in range(L)]

hamiltonian_map = [
    
    ["zz|",kin_energy],
    ["|n",mass_term],
    ["|I",mass_term],

    ["+|+-",interaction_term],
    ["-|-+",interaction_term]
]



Ham_LGT = hamiltonian(hamiltonian_map,dynamic_list = [],basis=basis,**no_checks)
Hamiltonian_LGT = Ham_LGT + 1e2*G
ham_matrix = Hamiltonian_LGT.toarray()
ham_LGT_matrix = Ham_LGT.toarray()

In [21]:
def check_hermitian(a, rtol=1e-10, atol=1e-10):
    return np.allclose(a, a.conj().T, rtol=rtol, atol=atol)

In [22]:
print(check_hermitian(ham_matrix))

True


In [23]:
print(check_hermitian(ham_LGT_matrix))

True


In [24]:
eigenval,eigenvec = np.linalg.eigh(ham_LGT_matrix)

In [25]:
print(eigenval)

[-8.45513846e-01 -6.97181775e-01 -6.97181775e-01 -6.55523919e-01
 -5.65690520e-01 -5.65690520e-01 -5.41231717e-01 -5.07745045e-01
 -5.07745045e-01 -4.11937408e-01 -4.11937408e-01 -3.92563772e-01
 -3.48669983e-01 -3.48669983e-01 -2.47335381e-01 -2.47335381e-01
 -2.30995281e-01 -1.94863959e-01 -1.94863959e-01 -1.38986692e-01
 -1.38986692e-01 -1.31480855e-01 -1.31480855e-01 -7.84107607e-02
 -7.28052608e-02 -7.28052608e-02 -3.89866919e-02 -3.89866919e-02
 -4.89590464e-16 -3.63323735e-17  2.71583853e-02  2.71583853e-02
  5.92559358e-02  5.92559358e-02  5.92559358e-02  5.92559358e-02
  8.16574038e-02  8.16574038e-02  9.33655919e-02  9.33655919e-02
  9.33655919e-02  9.33655919e-02  1.27158385e-01  1.27158385e-01
  1.38163098e-01  1.38163098e-01  1.39672219e-01  1.39672219e-01
  1.39672219e-01  1.39672219e-01  1.53818556e-01  1.59255936e-01
  1.59255936e-01  1.59255936e-01  1.59255936e-01  1.77890580e-01
  1.77890580e-01  1.93365592e-01  1.93365592e-01  1.93365592e-01
  1.93365592e-01  2.22950

In [26]:
eigenvalues,eigenvectors = np.linalg.eigh(ham_matrix)

In [27]:
print(eigenvalues[eigenvalues<=1e1])

[-0.84551385  0.22748942  0.5         0.776925    1.29754183  2.04355759]


In [28]:
for i in range(basis.Ns):
    if np.abs(eigenvectors[:,3][i])>=1e-3:
        print(i)

3
53
70
201
218
268


In [29]:
print(eigenvectors[:,0])

[ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
 -2.87138791e-19+1.38853067e-18j -5.22557745e-03-3.93703388e-02j
  0.00000000e+00+0.00000000e+00j -1.79842230e-17+5.90891150e-17j
  0.00000000e+00+0.00000000e+00j  2.54615840e-57+0.00000000e+00j
  9.37278672e-33+7.78426696e-34j  9.49209437e-34-7.97110112e-34j
  3.12975824e-33+9.63867993e-33j  1.67014957e-32+9.24136197e-34j
 -2.34514242e-33+5.61994484e-34j -5.49705422e-30+1.21033052e-32j
  2.30547307e-27-4.91677493e-30j -4.76305525e-22-4.96676514e-22j
  1.41859470e-23+4.25800338e-24j -7.24331425e-24+1.07497936e-23j
 -8.01040711e-36+0.00000000e+00j  7.54733527e-19+7.85570646e-19j
  1.90758920e-19-5.86296773e-19j -6.50431162e-20+5.43011774e-20j
  3.16545242e-17-5.56670719e-18j  8.67236987e-19-2.54011587e-19j
 -4.81295442e-17-6.67776109e-17j -2.59198762e-17+7.27356747e-17j
 -1.16375264e-32+1.21815069e-32j -5.75408277e-17+8.88012392e-18j
 -1.33647457e-16+1.33122441e-17j -6.07975859e-19-6.46519783e-19j
  1.37097390e-32+2.335767